## <center>Missing data in supervised ML</center>
### <center>Andras Zsom</center>
<center>Lead Data Scientist and  Adjunct Lecturer in Data Science</center>
<center>Center for Computation and Visualization</center>
<center>Brown University</center>
<center>Providence, RI, USA</center>

## About me
- Born and raised in Hungary
- Astrophysics PhD at MPIA, Heidelberg, Germany
- Postdoctoral researcher at MIT (still in astrophysics at the time)
- Started at Brown in December 2015 as a Data Scientist
- Promoted to Lead Data Scientist in 2017
- Adjunct Lecturer in Data Science this semester
   - Teaching the course *DATA1030: Hands-on data science* to the DS master students at Brown

## Data Science at Brown
- Center for Computation and Visualization
- Institutional Data group
   - Data-driven decision support and predictive modeling for Brown’s administrative units
   - Academic research on data-intensive projects
- **OPEN POSITION** - more on this later

## Learning Objectives

By the end of this workshop, you will be able to
- Describe the three main types of missingness patterns
- Evaluate simple approaches for handling missing values
- Apply multivariate imputation
- Apply XGBoost to a dataset with missing values
- Apply the reduced-features model (also called the pattern submodel approach)
- Decide which approach is best for your dataset

## Before we start, a few words on our dataset: kaggle house price
- good for educational purposes
   - messy data that requires quite a bit of preprocessing
   - a nice mixture of continuous, ordinal, and categorical features, each feature type has missing values
- lots of excellent kernels on kaggle
   - check them out [here]()
- dataset and description available in repo
   - let's take a look!

## <font color='LIGHTGRAY'>Learning Objectives</font>

<font color='LIGHTGRAY'>By the end of this workshop, you will be able to</font>
- **Describe the three main types of missingness patterns**
- <font color='LIGHTGRAY'>Evaluate simple approaches for handling missing values</font>
- <font color='LIGHTGRAY'>Apply multivariate imputation</font>
- <font color='LIGHTGRAY'>Apply XGBoost to a dataset with missing values</font>
- <font color='LIGHTGRAY'>Apply the reduced-features model (also called the pattern submodel approach)</font>
- <font color='LIGHTGRAY'>Decide which approach is best for your dataset</font>

## Missing values often occur in datasets
- survey data: not everyone answers all the questions
- medical data: not all tests/treatments/etc are performed on all patients
- sensor can be offline or malfunctioning
- customer data: not every user uses all features of an app

## Missing values are an issue for multiple reasons

#### Concenptual reason
- missing values can introduce biases
    - bias: the samples (the data points) are not representative of the underlying distribution/population
    - any conclusion drawn from a biased dataset is also biased.
    - rich people tend to not fill out survey questions about their salaries and the mean salary estimated from survey data tend to be lower than true value


#### Practical reason
- missing values (NaN, NA, inf) are incompatible with sklearn
   - all values in an array need to be numerical otherwise sklearn will throw a *ValueError*
- there are a few supervised ML techniques that work with missing values (e.g., XGBoost, CatBoost)
   - we will cover those later today

# Missing data patterns

- **MCAR** - Missing Complete At Random
   - some people skip some survey questions by accident
- **MAR** - Missing At Random
   - males are less likely to fill out a survey on depression
   - this has nothing to do with their level of depression after accounting for maleness
- **MNAR** - Missing Not At Random
   - depressed people are less likely to fill out a survey on depression due to their level of depression

## MCAR test

- MCAR can be diagnosed with a statistical test ([Little, 1988](https://www.tandfonline.com/doi/abs/10.1080/01621459.1988.10478722))
   - python implementation available in the [pymice](https://github.com/RianneSchouten/pymice) package or in the skipped slide

In [1]:
# from the pymice package 
# https://github.com/RianneSchouten/pymice

import numpy as np
import pandas as pd
import math as ma
import scipy.stats as st

def checks_input_mcar_tests(data):
    """ Checks whether the input parameter of class McarTests is correct
            Parameters
            ----------
            data:
                The input of McarTests specified as 'data'
            Returns
            -------
            bool
                True if input is correct
            """

    if not isinstance(data, pd.DataFrame):
        print("Error: Data should be a Pandas DataFrame")
        return False

    if not any(data.dtypes.values == np.float):
        if not any(data.dtypes.values == np.int):
            print("Error: Dataset cannot contain other value types than floats and/or integers")
            return False

    if not data.isnull().values.any():
        print("Error: No NaN's in given data")
        return False

    return True


def mcar_test(data):
    """ Implementation of Little's MCAR test
    Parameters
    ----------
    data: Pandas DataFrame
        An incomplete dataset with samples as index and variables as columns
    Returns
    -------
    p_value: Float
        This value is the outcome of a chi-square statistical test, testing whether the null hypothesis
        'the missingness mechanism of the incomplete dataset is MCAR' can be rejected.
    """

    if not checks_input_mcar_tests(data):
        raise Exception("Input not correct")

    dataset = data.copy()
    vars = dataset.dtypes.index.values
    n_var = dataset.shape[1]

    # mean and covariance estimates
    # ideally, this is done with a maximum likelihood estimator
    gmean = dataset.mean()
    gcov = dataset.cov()

    # set up missing data patterns
    r = 1 * dataset.isnull()
    mdp = np.dot(r, list(map(lambda x: ma.pow(2, x), range(n_var))))
    sorted_mdp = sorted(np.unique(mdp))
    n_pat = len(sorted_mdp)
    correct_mdp = list(map(lambda x: sorted_mdp.index(x), mdp))
    dataset['mdp'] = pd.Series(correct_mdp, index=dataset.index)

    # calculate statistic and df
    pj = 0
    d2 = 0
    for i in range(n_pat):
        dataset_temp = dataset.loc[dataset['mdp'] == i, vars]
        select_vars = ~dataset_temp.isnull().any()
        pj += np.sum(select_vars)
        select_vars = vars[select_vars]
        means = dataset_temp[select_vars].mean() - gmean[select_vars]
        select_cov = gcov.loc[select_vars, select_vars]
        mj = len(dataset_temp)
        parta = np.dot(means.T, np.linalg.solve(select_cov, np.identity(select_cov.shape[1])))
        d2 += mj * (np.dot(parta, means))

    df = pj - n_var

    # perform test and save output
    p_value = 1 - st.chi2.cdf(d2, df)

    return p_value

## Takeaway
- it can be challenging to infer the missingness pattern from an incomplete dataset
   - There is a statistical test to diagnose MCAR
   - MAR and MNAR are difficult/impossible to diagnose to the best of my knowledge
- multiple patterns can be present in the data
   - even worse, multiple patterns can be present in one feature!
   - missing values in a feature can occur due to a mix of MCAR, MAR, MNAR 


## <font color='LIGHTGRAY'>Learning Objectives</font>

<font color='LIGHTGRAY'>By the end of this workshop, you will be able to</font>
- <font color='LIGHTGRAY'>Describe the three main types of missingness patterns</font>
- **Evaluate simple approaches for handling missing values**
- <font color='LIGHTGRAY'>Apply multivariate imputation</font>
- <font color='LIGHTGRAY'>Apply XGBoost to a dataset with missing values</font>
- <font color='LIGHTGRAY'>Apply the reduced-features model (also called the pattern submodel approach)</font>
- <font color='LIGHTGRAY'>Decide which approach is best for your dataset</font>

## Simple approaches for handling missing values
- 1) categorical/ordinal features: treat missing values as another category
    - missing values in categorical/ordinal features are not a big deal
- 2) continuous features: this is the tough part
    - sklearn's SimpleImputer
- 3) exclude points or features with missing values
    - might be OK

### 1a) Missing values in a categorical feature
- YAY - this is not an issue at all!
- Categorical feature needs to be one-hot encoded anyway
- Just replace the missing values with 'NA' or 'missing' and treat it as a separate category

### 1b) Missing values in a ordinal feature
- this can be a bit trickier but usually fine
- Ordinal encoder is applied to ordinal features
    - where does 'NA' or 'missing' fit into the order of the categories?
    - usually first or last
- if you can figure this out, you are golden

In [2]:
# read the data
import pandas as pd
import numpy  as np
from sklearn.model_selection import train_test_split

# Let's load the data
df = pd.read_csv('data/train.csv')
# drop the ID
df.drop(columns=['Id'],inplace=True)

# the target variable
y = df['SalePrice']
df.drop(columns=['SalePrice'],inplace=True)
# the unprocessed feature matrix
X = df.values
print(X.shape)
# the feature names
ftrs = df.columns

(1460, 79)


In [3]:
# let's split to train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train = pd.DataFrame(data=X_train,columns=ftrs)
X_test = pd.DataFrame(data=X_test,columns=ftrs)
print(X_train.head())

  MSSubClass MSZoning LotFrontage LotArea Street Alley LotShape LandContour  \
0         20       RL          90   11694   Pave   NaN      Reg         Lvl   
1         20       RL          60    6600   Pave   NaN      Reg         Lvl   
2         30       RL          80   13360   Pave  Grvl      IR1         HLS   
3         20       RL         NaN   13265   Pave   NaN      IR1         Lvl   
4         20       RL         118   13704   Pave   NaN      IR1         Lvl   

  Utilities LotConfig  ... ScreenPorch PoolArea PoolQC Fence MiscFeature  \
0    AllPub    Inside  ...         260        0    NaN   NaN         NaN   
1    AllPub    Inside  ...           0        0    NaN   NaN         NaN   
2    AllPub    Inside  ...           0        0    NaN   NaN         NaN   
3    AllPub   CulDSac  ...           0        0    NaN   NaN         NaN   
4    AllPub    Corner  ...           0        0    NaN   NaN         NaN   

  MiscVal MoSold YrSold SaleType SaleCondition  
0       0      7   

In [4]:
# collect the various features
cat_ftrs = ['MSZoning','Street','Alley','LandContour','LotConfig','Neighborhood','Condition1','Condition2',\
            'BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','Foundation',\
           'Heating','CentralAir','Electrical','GarageType','PavedDrive','MiscFeature','SaleType','SaleCondition']
ordinal_ftrs = ['LotShape','Utilities','LandSlope','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure',\
               'BsmtFinType1','BsmtFinType2','HeatingQC','KitchenQual','Functional','FireplaceQu','GarageFinish',\
               'GarageQual','GarageCond','PoolQC','Fence']
ordinal_cats = [['Reg','IR1','IR2','IR3'],['AllPub','NoSewr','NoSeWa','ELO'],['Gtl','Mod','Sev'],\
               ['Po','Fa','TA','Gd','Ex'],['Po','Fa','TA','Gd','Ex'],['NA','Po','Fa','TA','Gd','Ex'],\
               ['NA','Po','Fa','TA','Gd','Ex'],['NA','No','Mn','Av','Gd'],['NA','Unf','LwQ','Rec','BLQ','ALQ','GLQ'],\
               ['NA','Unf','LwQ','Rec','BLQ','ALQ','GLQ'],['Po','Fa','TA','Gd','Ex'],['Po','Fa','TA','Gd','Ex'],\
               ['Sal','Sev','Maj2','Maj1','Mod','Min2','Min1','Typ'],['NA','Po','Fa','TA','Gd','Ex'],\
               ['NA','Unf','RFn','Fin'],['NA','Po','Fa','TA','Gd','Ex'],['NA','Po','Fa','TA','Gd','Ex'],
               ['NA','Fa','TA','Gd','Ex'],['NA','MnWw','GdWo','MnPrv','GdPrv']]
num_ftrs = ['MSSubClass','LotFrontage','LotArea','OverallQual','OverallCond','YearBuilt','YearRemodAdd',\
             'MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF',\
             'LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr',\
             'KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageYrBlt','GarageCars','GarageArea','WoodDeckSF',\
             'OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal','MoSold','YrSold']

In [5]:
# preprocess with pipeline and columntransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# one-hot encoder
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant',fill_value='missing')),
    ('onehot', OneHotEncoder(sparse=False,handle_unknown='ignore'))])

# ordinal encoder
ordinal_transformer = Pipeline(steps=[
    ('imputer2', SimpleImputer(strategy='constant',fill_value='NA')),
    ('ordinal', OrdinalEncoder(categories = ordinal_cats))])

# standard scaler
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

# collect the transformers
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_ftrs),
        ('cat', categorical_transformer, cat_ftrs),
        ('ord', ordinal_transformer, ordinal_ftrs)])

In [6]:
# fit_transform the data
X_prep = preprocessor.fit_transform(X_train)
# little hacky, but collect feature names
feature_names = preprocessor.transformers_[0][-1] + \
                list(preprocessor.named_transformers_['cat'][1].get_feature_names(cat_ftrs)) + \
                preprocessor.transformers_[2][-1]

df_prep = pd.DataFrame(data=X_prep,columns=feature_names)
df_prep['SalePrice'] = y_train

print(df_prep.shape)

df_prep.to_csv('data/house_price_prep.csv',index=False)

df_test = preprocessor.transform(X_test)
df_test = pd.DataFrame(data=df_test,columns = feature_names)
df_test['SalePrice'] = y_test

print(df_test.shape)

df = pd.concat([df_prep,df_test],axis=0)
print(df.shape)

(1168, 226)
(292, 226)
(1460, 226)


### Let's take a closer look at our missing values!

In [7]:
print('p value of the mcar test:',mcar_test(df))
print('data dimensions:',df_prep.shape)
perc_missing_per_ftr = df_prep.isnull().sum(axis=0)/df_prep.shape[0]
print('fraction of missing values in features:')
print(perc_missing_per_ftr[perc_missing_per_ftr > 0])
frac_missing = sum(df_prep.isnull().sum(axis=1)!=0)/df_prep.shape[0]
print('fraction of points with missing values:',frac_missing)


p value of the mcar test: 0.0
data dimensions: (1168, 226)
fraction of missing values in features:
LotFrontage    0.181507
MasVnrArea     0.005137
GarageYrBlt    0.049658
SalePrice      0.206336
dtype: float64
fraction of points with missing values: 0.3895547945205479


### 2) Continuous features: mean or median imputation
- Imputation means you infer the missing values from the known part of the data
- sklearn's SimpleImputer can do mean and median imputation
- USUALLY A BAD IDEA!
   - MCAR: mean/median of non-missing values is the same as the mean/median of the true underlying distribution, but the variances are different
   - not MCAR: the mean/median and the variance of the completed dataset will be off
   - supervised ML model is too confident (MCAR) or systematically off (not MCAR)

### 3) Exclude points or features with missing values

- easy to do with pandas
- it is an ACCEPTABLE approach under two conditions:
    - Little's test supports MCAR (p > 0.05)
    - only small fraction of points contain missing values (maybe a few percent?)
    - the missing values are limited to one or a few features and a large fraction of points are missing from those features (maybe up to 90%?)
- if the MCAR assumption is justified, dropping points will not introduce biases to your model
- due to the smaller sample size, the confidence of your model might suffer. 
- what will you do with missing values when you deploy the model?

In [8]:
print(df_prep.shape)
# by default, rows/points are dropped
df_r = df_prep.dropna()
print(df_r.shape)
# drop features with missing values
df_c = df_prep.dropna(axis=1)
print(df_c.shape)


(1168, 226)
(713, 226)
(1168, 222)


## <font color='LIGHTGRAY'>Learning Objectives</font>

<font color='LIGHTGRAY'>By the end of this workshop, you will be able to</font>
- <font color='LIGHTGRAY'>Describe the three main types of missingness patterns</font>
- <font color='LIGHTGRAY'>Evaluate simple approaches for handling missing values</font>
- **Apply multivariate imputation**
- <font color='LIGHTGRAY'>Apply XGBoost to a dataset with missing values</font>
- <font color='LIGHTGRAY'>Apply the reduced-features model (also called the pattern submodel approach)</font>
- <font color='LIGHTGRAY'>Decide which approach is best for your dataset</font>

## sklearn's IterativeImputer



## <font color='LIGHTGRAY'>Learning Objectives</font>

<font color='LIGHTGRAY'>By the end of this workshop, you will be able to</font>
- <font color='LIGHTGRAY'>Describe the three main types of missingness patterns</font>
- <font color='LIGHTGRAY'>Evaluate simple approaches for handling missing values</font>
- <font color='LIGHTGRAY'>Apply multivariate imputation</font>
- **Apply XGBoost to a dataset with missing values**
- <font color='LIGHTGRAY'>Apply the reduced-features model (also called the pattern submodel approach)</font>
- <font color='LIGHTGRAY'>Decide which approach is best for your dataset</font>

## XGBoost and missing values
- sklearn assumes the feature matrix (X) and the target variable (y) are complete 
- XGBoost doesn't! 
   - it runs just fine with np.nans in X

In [9]:
help(OneHotEncoder)

Help on class OneHotEncoder in module sklearn.preprocessing._encoders:

class OneHotEncoder(_BaseEncoder)
 |  Encode categorical integer features as a one-hot numeric array.
 |  
 |  The input to this transformer should be an array-like of integers or
 |  strings, denoting the values taken on by categorical (discrete) features.
 |  The features are encoded using a one-hot (aka 'one-of-K' or 'dummy')
 |  encoding scheme. This creates a binary column for each category and
 |  returns a sparse matrix or dense array.
 |  
 |  By default, the encoder derives the categories based on the unique values
 |  in each feature. Alternatively, you can also specify the `categories`
 |  manually.
 |  The OneHotEncoder previously assumed that the input features take on
 |  values in the range [0, max(values)). This behaviour is deprecated.
 |  
 |  This encoding is needed for feeding categorical data to many scikit-learn
 |  estimators, notably linear models and SVMs with the standard kernels.
 |  
 | 

## <font color='LIGHTGRAY'>Learning Objectives</font>

<font color='LIGHTGRAY'>By the end of this workshop, you will be able to</font>
- <font color='LIGHTGRAY'>Describe the three main types of missingness patterns</font>
- <font color='LIGHTGRAY'>Evaluate simple approaches for handling missing values</font>
- <font color='LIGHTGRAY'>Apply multivariate imputation</font>
- <font color='LIGHTGRAY'>Apply XGBoost to a dataset with missing values</font>
- **Apply the reduced-features model (also called the pattern submodel approach)**
- <font color='LIGHTGRAY'>Decide which approach is best for your dataset</font>

## <font color='LIGHTGRAY'>Learning Objectives</font>

<font color='LIGHTGRAY'>By the end of this workshop, you will be able to</font>
- <font color='LIGHTGRAY'>Describe the three main types of missingness patterns</font>
- <font color='LIGHTGRAY'>Evaluate simple approaches for handling missing values</font>
- <font color='LIGHTGRAY'>Apply multivariate imputation</font>
- <font color='LIGHTGRAY'>Apply XGBoost to a dataset with missing values</font>
- <font color='LIGHTGRAY'>Apply the reduced-features model (also called the pattern submodel approach)</font>
- **Decide which approach is best for your dataset**

Now you can
- Describe the three main types of missingness patterns
- Evaluate simple approaches for handling missing values
- Apply multivariate imputation
- Apply XGBoost to a dataset with missing values
- Apply the reduced-features model (also called the pattern submodel approach)
- Decide which approach is best for your dataset

### Thanks for you attention!